## Specific Test IV. Exploring Equivariant Neural Networks

My thoughts on using CNN
- The number of parameters in a neural network increases rapidly in relation to the number of layers and hence make the model computationally heavy considering the fact that we are using pixel heavy images. Tuning so many of parameters can be time consuming. This is the main reason why I have used CNN for this task.
- The lensing images have high dimensionality (each pixel considered as a feature) which suits the above described abilities of CNNs. They are trained to identify the edges of objects in any image.
- The pre-processing required in a CNN is much lower as compared to other classification algorithms. While in primitive methods filters are hand-engineered, with enough training, CNNs have the ability to learn these filters/characteristics.
- This is my first time using CNN with pytorch as I commonly use tensorflow for CNNs so I was unable to plot the ROC curve as I ran out of time.

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import cv2
import numpy as np
from tqdm import tqdm


REBUILD_DATA = True 

class DM_Classifier():
    IMG_SIZE = 50
    sub = "lenses/sub"
    no_sub = "lenses/no_sub"
    
    LABELS = {sub: 0, no_sub: 1}
    training_data = []

    subcount = 0
    nosubcount = 0

    def make_training_data(self):
        for label in self.LABELS:
            print(label)
            for f in tqdm(os.listdir(label)):
                if "jpg" in f:
                    try:
                        path = os.path.join(label, f)
                        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                        img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
                        self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])   
                        

                        if label == self.HEELS:
                            self.subcount += 1
                        elif label == self.no_sub:
                            self.nosubcount += 1

                    except Exception as e:
                        pass
                        

        np.random.shuffle(self.training_data)
        np.save("training_data.npy", self.training_data)
        print('sub:',dmclassifier.subcount)
        print('no sub:',dmclassifier.nosubcount)

if REBUILD_DATA:
    dmclassifier = DM_Classifier()
    dmclassifier.make_training_data()
            

training_data = np.load("training_data.npy", allow_pickle=True)
print(len(training_data))
        

lenses/sub


  0%|          | 16/5000 [00:00<00:31, 159.10it/s]

lenses/no_sub


100%|██████████| 5000/5000 [00:22<00:00, 221.43it/s]


sub: 0
no sub: 0
10000


In [11]:
class Net(nn.Module):
    def __init__(self):
        super().__init__() 
        self.conv1 = nn.Conv2d(1, 32, 5) # input is an image with 32 output channels and 5x5 kernel
        self.conv2 = nn.Conv2d(32, 64, 5) # input is 32, bc the first layer output 32, then later to 64 channels, 5x5 kernel / window
        self.conv3 = nn.Conv2d(64, 128, 5)

        x = torch.randn(50,50).view(-1,1,50,50)
        self._to_linear = None
        self.convs(x)

        self.fc1 = nn.Linear(self._to_linear, 512) #Flattening.
        self.fc2 = nn.Linear(512, 2) 

    def convs(self, x):
        
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))

        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x

    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)  #reshape
        x = F.relu(self.fc1(x))
        x = self.fc2(x)     #Output layer
        return F.softmax(x, dim=1)

In [12]:
net = Net()

optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.MSELoss()

X = torch.Tensor([i[0] for i in training_data])
X = X/225.0
y = torch.Tensor([i[1] for i in training_data])

VAL_PCT = 0.1  # validation
val_size = int(len(X)*VAL_PCT)


X_train = X[:-val_size]
y_train = y[:-val_size]

X_test = X[-val_size:]
y_test = y[-val_size:]
print(len(X_train), len(X_test))

BATCH_SIZE = 100
EPOCHS = 10

for epoch in range(EPOCHS):
    for i in tqdm(range(0, len(X_train), BATCH_SIZE)): 
    
        batch_X = X_train[i:i+BATCH_SIZE].view(-1, 1, 50, 50)
        batch_y = y_train[i:i+BATCH_SIZE]

        net.zero_grad()

        outputs = net(batch_X)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()   

    print(f"Epoch: {epoch}. Loss: {loss}")

    
correct = 0
total = 0
with torch.no_grad():
    for i in tqdm(range(len(X_test))):
        real_class = torch.argmax(y_test[i])
        net_out = net(X_test[i].view(-1, 1, 50, 50))[0]  #Returns List 
        predicted_class = torch.argmax(net_out)

        if predicted_class == real_class:
            correct += 1
        total += 1
print("Accuracy: ", round(correct/total, 3))

  0%|          | 0/90 [00:00<?, ?it/s]

9000 1000


  0%|          | 0/90 [00:00<?, ?it/s]

Epoch: 0. Loss: 0.24959498643875122


  0%|          | 0/90 [00:00<?, ?it/s]

...

Epoch: 8. Loss: 0.022373860701918602


  0%|          | 4/1000 [00:00<00:25, 39.69it/s]

Epoch: 9. Loss: 0.018530042842030525


100%|██████████| 1000/1000 [00:02<00:00, 347.23it/s]

Accuracy:  0.972


In [13]:
X_test

tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.0044, 0.0044, 0.0044],
         [0.0000, 0.0000, 0.0000,  ..., 0.0044, 0.0044, 0.0044],
         [0.0000, 0.0000, 0.0000,  ..., 0.0044, 0.0044, 0.0044],
         ...,
         [0.0044, 0.0044, 0.0044,  ..., 0.0000, 0.0044, 0.0044],
         [0.0044, 0.0044, 0.0044,  ..., 0.0044, 0.0044, 0.0044],
         [0.0044, 0.0044, 0.0044,  ..., 0.0044, 0.0044, 0.0044]],

        [[0.0089, 0.0089, 0.0089,  ..., 0.0089, 0.0044, 0.0089],
         [0.0089, 0.0089, 0.0089,  ..., 0.0089, 0.0044, 0.0089],
         [0.0089, 0.0089, 0.0089,  ..., 0.0089, 0.0222, 0.0044],
         ...,
         [0.0089, 0.0089, 0.0089,  ..., 0.0044, 0.0000, 0.0044],
         [0.0089, 0.0089, 0.0089,  ..., 0.0089, 0.0044, 0.0044],
         [0.0089, 0.0089, 0.0089,  ..., 0.0089, 0.0044, 0.0044]],

        [[0.0044, 0.0044, 0.0044,  ..., 0.0044, 0.0044, 0.0044],
         [0.0044, 0.0044, 0.0044,  ..., 0.0044, 0.0044, 0.0044],
         [0.0044, 0.0044, 0.0044,  ..., 0.0044, 0.0044, 0.

In [ ]:
predict_out = net(X_test)
predict_y = torch.max(predict_out, 1)
y_score= net(torch.Tensor(X_test))

fpr, tpr, thresholds = metrics.roc_curve(y_test, y_score, pos_label=2)
roc_auc = auc(fpr, tpr)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()


For more info about me, please visit my LinkedIn profile at https://www.linkedin.com/in/gaurav-g-s-9a7495180/ or feel free to drop a mail at ngs.gaurav.anaxagoras7@gmail.com